In [1]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import ResNet50
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
# Set paths
base_path = 'G://Chessman//Chess'
categories = ['Bishop', 'King', 'Knight', 'Pawn', 'Queen', 'Rook']
img_size = 128

In [3]:
# Load images and labels
data = []
labels = []

In [4]:
def load_images_from_folder(folder, label):
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        if img is not None:
            img_resized = cv2.resize(img, (img_size, img_size))
            data.append(img_resized)
            labels.append(label)

In [5]:
for category in categories:
    folder_path = os.path.join(base_path, category)
    label = categories.index(category)
    load_images_from_folder(folder_path, label)

In [6]:
# Convert to numpy arrays
data = np.array(data) / 255.0  # Normalize
labels = np.array(labels)

In [7]:
# Convert labels to one-hot encoding
labels = to_categorical(labels, num_classes=len(categories))

In [8]:
# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

print(f'Training set shape: {X_train.shape}, Validation set shape: {X_val.shape}')

Training set shape: (444, 128, 128, 3), Validation set shape: (111, 128, 128, 3)


In [9]:
# Load the pre-trained ResNet50 model, excluding the top layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 55s 1us/step


In [10]:
# Add custom classification layers
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(len(categories), activation='softmax')(x)

In [11]:
# Final model
model = Model(inputs=base_model.input, outputs=output)

In [12]:
# Freeze the base layers (to retain the pre-trained knowledge)
for layer in base_model.layers:
    layer.trainable = False

In [13]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
# Model summary
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 134, 134,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 64, 64,    │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 64, 64,    │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 64, 64,    │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 66, 66,    │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 32, 32,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 32, 32,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 32, 32,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 32, 32,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 32, 32,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 32, 32,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 32, 32,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 32, 32,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 32, 32,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 32, 32,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 32, 32,    │      1,024 │ conv2_block1_3_c

 Total params: 40,368,518 (153.99 MB)

 Trainable params: 16,780,806 (64.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [16]:
# Callbacks with the new `.keras` format
checkpoint = ModelCheckpoint('best_chess_model.keras', monitor='val_accuracy', save_best_only=True, verbose=1)
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)

In [18]:
# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32,
    callbacks=[checkpoint, early_stopping]
)

Epoch 1/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4918 - loss: 1.2651
Epoch 1: val_accuracy improved from 0.45045 to 0.48649, saving model to best_chess_model.keras
14/14 ━━━━━━━━━━━━━━━━━━━━ 27s 2s/step - accuracy: 0.4910 - loss: 1.2667 - val_accuracy: 0.4865 - val_loss: 1.4243
Epoch 2/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4818 - loss: 1.3004
Epoch 2: val_accuracy did not improve from 0.48649
14/14 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - accuracy: 0.4821 - loss: 1.2984 - val_accuracy: 0.4324 - val_loss: 1.4450
Epoch 3/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5074 - loss: 1.2012
Epoch 3: val_accuracy did not improve from 0.48649
14/14 ━━━━━━━━━━━━━━━━━━━━ 42s 2s/step - accuracy: 0.5063 - loss: 1.2035 - val_accuracy: 0.4414 - val_loss: 1.4414
Epoch 4/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4807 - loss: 1.2813
Epoch 4: val_accuracy did not improve from 0.48649
14/14 ━━━━━━━━━━━━━━━━━━━━ 24s 2s/step - accuracy: 0.4807 - loss: 1.2813 - 

In [19]:
# Save the final model
model.save('final_chess_model.h5')

In [21]:
best_model = load_model('best_chess_model.keras')

# Evaluate on validation data
val_loss, val_acc = best_model.evaluate(X_val, y_val)
print(f'Validation Accuracy: {val_acc * 100:.2f}%')

4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.4811 - loss: 1.4391
Validation Accuracy: 48.65%
